# Bayesian Cognitive Modelling (Heuristics)

## Introduction

Take-the-best (TTB; Gigerenzer & Goldstein, 1996) is a heuristic model for human decision making, most notably for inferential tasks.

Consider a question in which an individual needs to choose one out of two stimuli as their answer. A classic example is the "German cities task", for instance, "which of the two following cities has a larger population, Frankfurt or Munich?". The TTB heuristic assumes that an individual evaluates the two stimuli based on a number of cues: in the case of the German cities task, cues are characteristics of cities such as whether one of them is the capital, whether they have a football team in the league, etc.

Cues are associated with cue validity values, each of which represent how often the presence of that cue in only one stimulus leads to the correct answer.

The most basic form of the TTB heuristic assumes that decision makers search the cues, one by one, based on cue validity values in a descending order. It further assumes that individuals stop the search immediately when a discriminating cue is found, that is, a cue which is present in only one stimuli. If a discriminating cue is present, they will choose the stimuli which has the cue. Otherwise, they will choose randomly.

This work is done in JAGS and is based on examples from Lee & Wagenmakers (2013), with modifications.

In [1]:
library(R.matlab)
library(data.table)
library(magrittr)
library(rjags)

R.matlab v3.6.2 (2018-09-26) successfully loaded. See ?R.matlab for help.


Attaching package: ‘R.matlab’


The following objects are masked from ‘package:base’:

    getOption, isOpen


Loading required package: coda

Linked to JAGS 4.3.0

Loaded modules: basemod,bugs



In [2]:
dat_heuristics<-readMat("Data/Data_Heuristics.mat") # loads data from MATLAB format

In [3]:
# saves data as matrices, these will be used in modelling

mat_decisions<-dat_heuristics$y
mat_stimuli<-dat_heuristics$m
mat_questions<-dat_heuristics$p
vec_cues<-dat_heuristics$v %>% as.numeric

In [4]:
# further saves data as data frames, for inspection

dat_decisions<-mat_decisions %>% as.data.table
dat_stimuli<-mat_stimuli %>% as.data.table
dat_questions<-mat_questions %>% as.data.table
dat_cues<-vec_cues %>% as.data.table

In [5]:
# renames columns for the data frames

dat_decisions<-cbind(rownames(dat_decisions), dat_decisions)
colnames(dat_decisions)<-c("p_id", paste0("Q", seq_len(length(dat_decisions)-1)))

dat_stimuli<-cbind(rownames(dat_stimuli), dat_stimuli)
colnames(dat_stimuli)<-c("Stimulus", paste0("Cue_", seq_len(length(dat_stimuli)-1)))

dat_questions<-cbind(rownames(dat_questions), dat_questions)
colnames(dat_questions)<-c("Question", "Stimulus_A", "Stimulus_B")

dat_cues<-cbind(rownames(dat_cues), dat_cues)
colnames(dat_cues)<-c("Cue", "Validity")

## Data Description

The data is taken from Newell and colleagues (2003; 2011).

### Stimuli

83 German cities were used as stimuli, while 9 cues were identified by the researchers.

Below, 1 means a cue is present for a stimulus, 0 means absent.

In [6]:
head(dat_stimuli)

Stimulus,Cue_1,Cue_2,Cue_3,Cue_4,Cue_5,Cue_6,Cue_7,Cue_8,Cue_9
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,0,1,1,1,1,0,0
2,0,1,1,0,1,1,1,0,0
3,0,1,1,1,1,1,1,0,0
4,0,1,1,1,1,1,0,0,0
5,0,1,1,1,1,1,0,0,0
6,0,1,0,1,1,1,0,0,1


The following are the validity values of the cues:

In [7]:
dat_cues

Cue,Validity
<chr>,<dbl>
1,1.0000
2,0.9643
3,0.9118
4,0.9000
5,0.8000
6,0.7843
7,0.6923
8,0.6538
9,0.5588


30 questions were included in the task, each of which has a pair of stimuli (i.e. cities) as potential answers:

In [8]:
head(dat_questions)

Question,Stimulus_A,Stimulus_B
<chr>,<dbl>,<dbl>
1,65,44
2,83,44
3,46,65
4,46,70
5,47,57
6,47,67


### Participants' Answers

There were 20 participants, each of whom were presented with all 30 questions.

Below, 1 represents the choice of stimulus A while 0 represents B.

In [9]:
head(dat_decisions)

p_id,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,⋯,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28,Q29,Q30
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,1,1,0,1,0,0,1,⋯,1,1,1,1,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,⋯,1,1,1,0,1,1,0,1,1,0
3,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,0,0,0,1,0
4,1,1,0,1,1,1,1,1,0,⋯,0,1,1,1,0,1,0,1,1,1
5,0,1,0,1,1,0,1,1,1,⋯,1,0,1,0,1,0,1,0,0,0
6,1,1,1,1,1,0,1,1,1,⋯,1,1,1,1,0,1,1,1,0,0


## Model 1

### Mathematical Model

While the ideas behind the model are simple, our first challenge is to define the model mathematically.

The deterministic component of the model is $t_q$, which is the TTB decision of question $q$. That is, given the stimulus pair of a question (i.e. stimuli A & B), their available cues, and cue validity values, which stimulus is preferred by the TTB model.

$$
t_q \leftarrow \text{TTB}(\mathbf{a_q}, \mathbf{b_q})
$$

where $\mathbf{a_q}$ and $\mathbf{b_q}$ are the cues of stimuli A and B in question $q$.

The probabilistic components of the model are as follows:

$$
y_{iq} = \begin{cases}
\text{Bernoulli}(\gamma), & t_q=a \\
\text{Bernoulli}(1 - \gamma), & t_q=b \\
\text{Bernoulli}(0.5), & \text{otherwise}
\end{cases}
$$

where $y_{iq}$ is the response of participant $i$ on question $q$. $y_{iq} = 1$ when A is chosen and $y_{iq} = 0$ when B is chosen.

$$
\gamma \sim \text{Uniform}(0.5, 1)
$$

$\gamma$ is expected to be above 0.5. In other words, when the TTB decision is A, the actual response is expected to be A (but not always). Conversely, when the TTB decision is B, the actual response is expected to be B.

### Computational Implementation

This implementation is based on Lee & Wagenmakers (2013), who provided an elegant yet under-explained solution.

First, the cue validity values are recoded as rank-based. Assuming that the cues are sorted in a descending order (which is the case in the data set), the cue validity vector $\mathbf{s} = [9, 8, 7, ..., 3, 2, 1]$ is created.

Second, for question $q$ and cue $c$, the following "cue score" is calculated:

$$
(a_{qc} - b_{qc}) \times 2^{(s_c - 1)}
$$

where $a_{qc}$ represents whether stimulus A in question $q$ has cue $c$, $a_{qc} = 1$ if so, $a_{qc} = 0$ if not. Similarly for $b_{qc}$.

Therefore, $(a_{qc} - b_{qc}) = 1$ when only A has that cue, -1 when only B has that cue, 0 when the cue is not discriminating.

$2^{(s_c - 1)}$ ensures that the first (note that cues are sorted in cue validity) discriminating cue will always dictate choice. For instance, the most valid cue has $s_c = 9$, $2^{(s_c - 1)} = 2^{(9 - 1)} = 256$, which is greater than $\sum 2^{(s_c - 1)}$ for all cues which are less valid, hence that cue will have the heaviest weight if it is present in one of the stimuli.

Afterwards, the total "cue score" of question $q$ is computed, which is the sum of the above "cue scores" across all nine cues:

$$
\sum^9_{c=1} (a_{qc} - b_{qc}) \times 2^{(s_c - 1)}
$$

This total "cue score" is positive if the TTB decision is A, negative if B, 0 if neither.

Finally, with a number of programming tricks (e.g. the `step()` function in JAGS), the total "cue score" of question $q$ is simplified to 3 if the TTB decision is A, 2 if neither, 1 if B.

With the simplified "cue scores"/TTB decision, the actual response is taken from the vector $\mathbf{t} = [1 - \gamma, 0.5, \gamma]$.

That is, if the simplified "cue scores"/TTB decision is 3 (i.e. A), the response is $t_3 = \gamma$. Similarly, if the simplified "cue scores"/TTB decision is 1 (i.e. B), the response is $t_1 = 1 - \gamma$.

In [10]:
# sets up constants

n_stimuli<-nrow(mat_stimuli)
n_cues<-length(vec_cues)
n_questions<-nrow(mat_questions)
n_participants<-nrow(mat_decisions)

In [11]:
string_model_1<-" model{

    for (question in 1:n_questions){
      
        for (i in 1:n_participants){

          mat_decisions[i, question]~dbern(ttb[t[question]])
        
        }
      
    }

    vec_ranks[1:n_cues]<-rank(vec_cues[1:n_cues]) # creates rank-based validity values, the most valid cue has a rank of 9, the least has 1
    
    for (question in 1:n_questions){
        
        for (cue in 1:n_cues){

            # the following line calculates the cue scores for each cue in each question
            # the first part returns, for each cue, 1 if only stimulus A has that cue, -1 if B, 0 if non-discriminating
            # the second part places heavier weights on more valid cues, the most valid cue has a weight of 2^8 = 256, the least has 2^0 = 1, so that the first discriminating cue will always dictate choices (since 256 > the sum of all other weights, even if all other cues are present)
            
          temp_1[question, cue]<-(mat_stimuli[mat_questions[question, 1], cue]-mat_stimuli[mat_questions[question, 2], cue])*pow(2, vec_ranks[cue]-1)
            
        }
    
        temp_2[question]<-sum(temp_1[question, 1:n_cues]) # sums the cue scores for each question, positive if favours A, negative if B, 0 if neither

        temp_3[question]<-step(temp_2[question]) - 1*step(-temp_2[question]) # returns 1 if the total cue score of a question favours A, -1 if B, 0 if neither
        
        t[question]<-temp_3[question] + 2 # returns 3 if temp_3 is 1 (i.e. favours A), 2 if neither, 1 if temp_3 is -1 (i.e. favours B)

    }

      ttb[1]<-1-gamma
      ttb[2]<-0.5
      ttb[3]<-gamma

      gamma~dunif(0.5,1) # prior for gamma

} "

In [12]:
list_data_model_1<-list(mat_decisions=mat_decisions, mat_stimuli=mat_stimuli, mat_questions=mat_questions, vec_cues=vec_cues, n_cues=n_cues, n_questions=n_questions, n_participants=n_participants)

Four chains were run for 5000 iterations, with a burn-in of 1000.

In [13]:
model_1<-jags.model(textConnection(string_model_1), data=list_data_model_1, n.chains=4)

Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 600
   Unobserved stochastic nodes: 1
   Total graph size: 1574

Initializing model



In [14]:
update(model_1, 1000)

In [15]:
simulation_model_1<-coda.samples(model=model_1, variable.names=c("gamma"), n.iter=5000)

In [16]:
summary(simulation_model_1)


Iterations = 2001:7000
Thinning interval = 1 
Number of chains = 4 
Sample size per chain = 5000 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

          Mean             SD       Naive SE Time-series SE 
     0.7526667      0.0175509      0.0001241      0.0001574 

2. Quantiles for each variable:

  2.5%    25%    50%    75%  97.5% 
0.7174 0.7409 0.7530 0.7646 0.7862 


Results showed that $\gamma$ has a posterior mean of .75 and a 95% credible interval of [.72, .79].

The above can be interpreted as individuals were reasonably accurate in "executing" the TTB heuristic, that is, to "convert" the TTB decisions to actual responses.

## Model 2

A common variant of the TTB heuristic is the Weighted ADDitive (WADD; Lee & Cummins, 2004) model.

TTB assumes that individuals make a decision as soon as a discriminating cue is found. On the other hand, WADD assumes that decision makers accumulate evidence for the two stimuli using information from all discriminating cues. Specifically, it is usually assumed that individuals transform the validity values of the cues into evidence values, sum those associated with discriminating cues, and choose the stimulus with the higher total evidence values.

The following is a latent mixture model, which assumes that an individual consistently uses either TTB or WADD to make decisions.

### Mathematical Model

Indicator $z_i$ shows which heuristic model was used by participant $i$.

$z_i = 1$ when participant $i$ used TTB, $z_i = 0$ for WADD.

$$
z_i \sim \text{Bernoulli}(\phi)
$$

$$
\phi \sim \text{Uniform}(0, 1)
$$

The deterministic component of the model is $t_{iq}$, which is the TTB/WADD decision of participant $i$ on question $q$:

$$
t_{iq} \leftarrow \begin{cases}
\text{TTB}(\mathbf{a_q}, \mathbf{b_q}), & z_i = 1 \\
\text{WADD}(\mathbf{a_q}, \mathbf{b_q}), & z_i = 0
\end{cases}
$$

where $\mathbf{a_q}$ and $\mathbf{b_q}$ are the cues of stimuli A and B in question $q$.

The response of participant $i$ on question $q$, $y_{iq}$, is very similar to that in Model 1:

$$
y_{iq} = \begin{cases}
\text{Bernoulli}(\gamma), & t_{iq}=a \\
\text{Bernoulli}(1 - \gamma), & t_{iq}=b \\
\text{Bernoulli}(0.5), & \text{otherwise}
\end{cases}
$$

$$
\gamma \sim \text{Uniform}(0.5, 1)
$$

### Computational Implementation

The TTB component of the mixture model is nearly identical to that in Model 1, except a participant indicator is now included.

For a participant $i$ using TTB, the "cue score" of cue $c$ in question $q$ is:

$$
(a_{qc} - b_{qc}) \times 2^{(s_c - 1)}
$$

where $s_c$ is the rank-based validity value of cue $c$.

On the other hand, for a participant $i$ using WADD, the "cue score" of cue $c$ in question $q$ is:

$$
(a_{qc} - b_{qc}) \times e_c
$$

where $e_c$ is the evidence value of cue $c$.

$e_c$ is calculated as the log odd of $v_c$, which is the validity value of cue $c$:

$$
e_c = \log(\frac{v_c}{1- v_c})
$$

The "cue scores" are then summed and transformed, as in Model 1, which reflect TTB/WADD decisions.

Similarly, the response of a participant $i$ on question $q$ is taken from the vector $\mathbf{t} = [1 - \gamma, 0.5, \gamma]$ based on their TTB/WADD decision.

In [17]:
vec_evidence<-log(vec_cues/(1-vec_cues))

vec_evidence[1]<-100 # the validity value of the first cue is 1 and the associated evidence value is Inf, this is to change that evidence value from Inf to a relatively big number

In [18]:
string_model_2<-" model{

    for (i in 1:n_participants){

        for (question in 1:n_questions){

            mat_decisions[i, question]~dbern(equals(z[i], 1)*ttb[t[i, question, 1]] + equals(z[i], 0)*ttb[t[i, question, 2]])

        }

    }

    vec_ranks[1:n_cues]<-rank(vec_cues[1:n_cues]) # rank-based validity values

    # below is TTB

    for (i in 1:n_participants){

        for (question in 1:n_questions){

            for (cue in 1:n_cues){
        
                temp_1[i, question, cue]<-(mat_stimuli[mat_questions[question, 1], cue]-mat_stimuli[mat_questions[question, 2], cue])*pow(2, vec_ranks[cue]-1)

            }

            temp_2[i, question]<-sum(temp_1[i, question, 1:n_cues])

            temp_3[i, question]<-step(temp_2[i, question]) - 1*step(-temp_2[i, question])

            t[i, question, 1]<-temp_3[i, question]+2
    
        }
    }

    # below is WADD

    for (i in 1:n_participants){

        for (question in 1:n_questions){

            for (cue in 1:n_cues){

                temp_4[i, question, cue]<-(mat_stimuli[mat_questions[question, 1], cue]-mat_stimuli[mat_questions[question, 2], cue])*vec_evidence[cue]

            }
      
            temp_5[i, question]<-sum(temp_4[i, question, 1:n_cues])

            temp_6[i, question]<-step(temp_5[i, question]) - 1*step(-temp_5[i, question])

            t[i, question, 2]<-temp_6[i, question]+2

        }

    }

    ttb[1]<-1-gamma
    ttb[2]<-0.5
    ttb[3]<-gamma
  
    for (i in 1:n_participants){

        z[i]~dbern(phi)

    }

    gamma~dunif(0.5,1) # priors for gamma

    phi~dbeta(1,1) # priors for phi

} "

In [19]:
list_data_model_2<-list(mat_decisions=mat_decisions, mat_stimuli=mat_stimuli, mat_questions=mat_questions, vec_cues=vec_cues, vec_evidence=vec_evidence, n_cues=n_cues, n_questions=n_questions, n_participants=n_participants)

Again, four chains were run for 5000 iterations, with a burn-in of 1000.

In [20]:
model_2<-jags.model(textConnection(string_model_2), data=list_data_model_2, n.chains=4)

Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 600
   Unobserved stochastic nodes: 22
   Total graph size: 3011

Initializing model



In [21]:
update(model_2, 1000)

In [22]:
simulation_model_2<-coda.samples(model=model_2, variable.names=c("gamma", "phi"), n.iter=5000)

In [23]:
summary(simulation_model_2)


Iterations = 2001:7000
Thinning interval = 1 
Number of chains = 4 
Sample size per chain = 5000 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

        Mean      SD  Naive SE Time-series SE
gamma 0.8001 0.01636 0.0001157      0.0001513
phi   0.4521 0.11675 0.0008256      0.0010262

2. Quantiles for each variable:

        2.5%    25%    50%    75%  97.5%
gamma 0.7679 0.7890 0.8003 0.8113 0.8315
phi   0.2340 0.3685 0.4502 0.5331 0.6837


Results showed a higher $\gamma$ in Model 2 than in Model 1. One possible explanation is that Model 2 is a better model and better reflects how individuals make decision.

$\phi$ has a posterior mean of .45 and a 95% credible interval of [.23, .68]. The posterior mean shows that less than half of all participants used the simple form of TTB, but note that the interval was fairly big.

## Model 3

Another variant of the TTB model allows individuals to search the cues in different orders, rather than strictly assuming decision makers search the cues based on validity values.

### Mathematical Model

Similar to the Models 1 and 2, the deterministic component of the model is $t_{iq}$ which is the TTB decision of participant $i$ on question $q$:

$$
t_{iq} \leftarrow \text{TTB}_{\mathbf{si}}(\mathbf{a_q}, \mathbf{b_q})
$$

$\mathbf{s_i}$ is an addition to the model which is a vector representing the search order of participant $i$:

$$
\mathbf{s_i} \sim \text{Uniform}((1, ..., 9), ..., (9, ..., 1))
$$

This assumes each search order is equally likely.

The response of participant $i$ on question $q$, $y_{iq}$, is identical to that in Model 2:

$$
y_{iq} = \begin{cases}
\text{Bernoulli}(\gamma), & t_{iq}=a \\
\text{Bernoulli}(1 - \gamma), & t_{iq}=b \\
\text{Bernoulli}(0.5), & \text{otherwise}
\end{cases}
$$

$$
\gamma \sim \text{Uniform}(0.5, 1)
$$

### Computational Implementation

The algorithm is very similar to that of Model 1, except the individual differences in cue search orders which are implemented as follows: the rank of cue $c$ for participant $i$ is $s_{iq}$:

$$
s_{iq} \sim \text{Normal}(0, 0.001)
$$

This assumes the same prior for each $s_{iq}$, which mimics the behaviours of the mathematical model.

In [24]:
string_model_3<-"model{   

    for (i in 1:n_participants){

        for (question in 1:n_questions){

            mat_decisions[i, question]~dbern(ttb[t[i, question]])

        }

    }

    for (i in 1:n_participants){

        for (question in 1:n_questions){

            for (cue in 1:n_cues){

                temp_1[i, question, cue]<-(mat_stimuli[mat_questions[question, 1], cue]-mat_stimuli[mat_questions[question, 2], cue])*pow(2, vec_ranks[i, cue]-1)

            }

            temp_2[i, question]<-sum(temp_1[i, question, 1:n_cues])

            temp_3[i, question]<-step(temp_2[i, question]) - 1*step(-temp_2[i, question])

            t[i, question]<-temp_3[i, question]+2

        }

        vec_ranks[i, 1:n_cues]<-rank(vec_ranks_temp[i, 1:n_cues])

        for (cue in 1:n_cues){

            vec_ranks_temp[i, cue]~dnorm(0, .001)

        }

    }

    ttb[1]<-1-gamma
    ttb[2]<-0.5
    ttb[3]<-gamma
  
    gamma~dunif(0.5, 1) # prior for gamma

} "

In [25]:
list_data_model_3<-list(mat_decisions=mat_decisions, mat_stimuli=mat_stimuli, mat_questions=mat_questions, n_cues=n_cues, n_questions=n_questions, n_participants=n_participants)

Again, four chains were run for 5000 iterations, with a burn-in of 1000.

In [26]:
# this took a while to run, so the results were saved on disk instead

# model_3<-jags.model(textConnection(string_model_3), data=list_data_model_3, n.chains=4)
# update(model_3, 1000)
# simulation_model_3<-coda.samples(model=model_3, variable.names=c("gamma"), n.iter=5000)

# save(simulation_model_3, file="Data/Objects_Bayesian_Heuristics_Model_3.RData")

In [27]:
load("Data/Objects_Bayesian_Heuristics_Model_3.RData")

In [28]:
summary(simulation_model_3)


Iterations = 2001:7000
Thinning interval = 1 
Number of chains = 4 
Sample size per chain = 5000 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

          Mean             SD       Naive SE Time-series SE 
     0.8350827      0.0165804      0.0001172      0.0001968 

2. Quantiles for each variable:

  2.5%    25%    50%    75%  97.5% 
0.8018 0.8241 0.8354 0.8465 0.8664 


Results showed that $\gamma$ is higher than both Models 1 and 2, which suggested that Model 3 might be a more appropriate model for the data.